In [1]:
import csv
import numpy as np
import sys
import os
from osgeo import gdal
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir,'..','src'))
from util import *

sys.path.append(os.path.join(current_dir,'..','params'))
from load_params import *



In [2]:
#obtention du tableau RVB à partir des 3 images satellites B2,B3,B4 
chemin = os.path.abspath("../image/image_sondage/Paris")

#génération des images pour le sondage
fichier = "Paris"
resolution = ['10m', '20m', '60m']
img = "2024-10-24-0000_2024-10-24-2359_Sentinel-2_L2A"
RVB_liste = []

def RVB(root_srcpath, path_name, prod_name):
    bnames = ['B04','B03','B02']
    
    # Obtenir le chemin en entier
    srcpath = os.path.join(root_srcpath,path_name)
    srcfile = os.path.join(srcpath,prod_name+'_'+bnames[0]+'_Raw.tiff')
    src = gdal.Open(srcfile)
    
    # Obtenir les dimensions de l'image
    nx = src.RasterXSize
    ny = src.RasterYSize
    src = None
    
    # Préparer la matrice de sortie
    out_matrix = np.empty((nx * ny, 3), dtype=float)
    
    # Charger les bandes et remplir la matrice
    for i, bname in enumerate(bnames):
        srcfile = os.path.join(srcpath, prod_name + '_' + bname + '_Raw.tiff')
        src = gdal.Open(srcfile)
                
        # Lire la bande entière 
        arr = src.GetRasterBand(1).ReadAsArray().astype(float) 
        src = None
        
        out_matrix[:, i] = arr.flatten()
        
    return out_matrix, nx, ny
               
for k in range(len(resolution)):
    dossier = os.path.join(chemin,fichier+'_'+resolution[k])
    RVB_, nx, ny = RVB(chemin, dossier, img)
    RVB_ = np.clip(RVB_, 0,1)
    RVB_liste.append(RVB_)
               
print(np.shape(RVB_liste[0]))

(1086180, 3)


In [3]:

#convertion de RVB en XYZ
T_8_new = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq8_new
T_8 = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq8
T_9 = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq9

def RVB_to_XYZ(T, A):
    B = A @ T
    return B

XYZ = []
equation = [T_8_new, T_9]
for i in range(len(equation)):
    L = []
    for k in range(len(RVB_liste)):   
        XYZ_ = RVB_to_XYZ(equation[i], RVB_liste[k])
        L.append(XYZ_)
    XYZ.append(L)

#print(np.shape(XYZ))
print(np.shape(XYZ[0][0]))


(1086180, 3)


In [4]:
# XYZ to sRVB

#standart Martix define by CIE
M = np.array([[3.240625,-1.537208,-0.498629],
             [-0.968931,1.875756,0.041518],
             [0.055710, -0.204021,1.056996]])

sRVB = []
for i in range(len(equation)):
    L = []
    for k in range(len(resolution)):
        sRVB_ = M@XYZ[i][k].T
        sRVB_ = np.clip(sRVB_, 0, 1)
        L.append(sRVB_)
    sRVB.append(L)
    
    
print(sRVB[0][0][0])


[0.05109833 0.08214302 0.09416723 ... 0.05834923 0.07315035 0.08387036]


In [8]:
#sRVB en sRVB 8bit et application gamma 2.4
def phi(R):
    for k in range(len(R)):
        if R[k] <= 0.0031308:
            R[k] = R[k]*65535*12.92
        else:
            R[k] = 65535*(1.055*R[k]**(1/2.4) - 0.055)
    return R

sRVB_16bit = []
print(sRVB[0][0][0])


for i in range(len(equation)): #on fait pour l'quation T9 et T8
    L = []
    for k in range(len(resolution)):#on fait pour toutes les résolutions
        R = sRVB[i][k][0]
        V = sRVB[i][k][1]
        B = sRVB[i][k][2]
        
        R_16bit = phi(R)
        V_16bit = phi(V)
        B_16bit = phi(B)
        
        sRVB_16bit_ = np.array([R_16bit,V_16bit,B_16bit]).T

        sRVB_16bit_ = np.clip(sRVB_16bit, 0, 65535).astype(np.uint16)
        L.append(sRVB_16bit_)
        
    sRVB_16bit.append(L)

print(np.shape(sRVB_16bit))
print('Ok')

[3942169.37845139 4350675.57371297 4472944.97437832 ... 4053920.48292769
 4248613.16689823 4369158.73285964]
(2, 3)
Ok


/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [6]:

#gdal en imageio
import imageio.v2 as imageio

for k in range(len(resolution)):
    #Matrice 16bits
    sRVB_img_T8 = sRVB_16bit[0][k].reshape(ny,nx,3)
    sRVB_img_T9 = sRVB_16bit[1][k].reshape(ny,nx,3)

    # Transformer en un format supporté par imageio (par exemple PNG)
    output_tiff_filename_T8 = "../image/image_sondage/T8/"+fichier+resolution[k]+"_T8_new_16bits.tiff"
    output_tiff_filename_T9 = "../image/image_sondage/T9/"+fichier+resolution[k]+"_T9_16bits.tiff"

    # Enregistrer l'image
    imageio.imwrite(output_tiff_filename_T8, sRVB_img_T8,  format='TIFF')
    imageio.imwrite(output_tiff_filename_T9, sRVB_img_T9,  format='TIFF')


[[array([], dtype=uint16), array([], dtype=uint16), array([], dtype=uint16)], [array([], shape=(1, 3, 0), dtype=uint16), array([], shape=(1, 3, 0), dtype=uint16), array([], shape=(1, 3, 0), dtype=uint16)]]


ValueError: cannot reshape array of size 0 into shape (140,215,3)